In [52]:
import asyncio
import chess
import chess.engine
import time
import duckdb
import pandas as pd

In [3]:
db = duckdb.connect("chess.duckdb")
db.sql("SET TimeZone = 'UTC'")

In [4]:
db.sql("select * from information_schema.tables")

┌───────────────┬─────────────────┬──────────────────────┬───┬──────────┬───────────────┬───────────────┐
│ table_catalog │  table_schema   │      table_name      │ … │ is_typed │ commit_action │ TABLE_COMMENT │
│    varchar    │     varchar     │       varchar        │   │ varchar  │    varchar    │    varchar    │
├───────────────┼─────────────────┼──────────────────────┼───┼──────────┼───────────────┼───────────────┤
│ chess         │ chess_data_prep │ game_moves           │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_prep │ game_moves_centipawn │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_prep │ player_games         │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_prep │ player_game_moves    │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_raw  │ players_games        │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_raw  │ players_pr

In [41]:
db.sql("select * from chess_data_prep.player_games where uuid = '0382938d-05f5-11ef-9338-b21a8d01000f' ").to_df()

,player_color,player_rating,player_result,opponent_rating,opponent_result,is_stronger_opponent,player_wdl,player_wdl_reason,eco,game_start_date,...,white__aid,white__username,white__uuid,black__rating,black__result,black__aid,black__username,black__uuid,accuracies__white,accuracies__black
0,Black,599,checkmated,622,win,True,lose,checkmated,B53,2024-04-29,...,https://api.chess.com/pub/player/huyhayden,HuyHayden,c507c630-42aa-11ee-808f-ddd1604df8bb,599,checkmated,https://api.chess.com/pub/player/johnnywhoopp,johnnywhoopp,a8d052da-f18b-11ee-b5a6-410d39dba314,NaN,NaN


In [45]:
q = """
with move as (
    select 
    uuid
    , game_move_index
    , game_move_fen 
    from chess_data_prep.game_moves 
    where uuid = '0382938d-05f5-11ef-9338-b21a8d01000f' 
    order by game_move_index
)
, player_color as (
    select
    uuid
    , player_color
    from chess_data_prep.player_games 
    where uuid = '0382938d-05f5-11ef-9338-b21a8d01000f'
)
select
player_color
, m.*
from move as m
left join player_color as pc
on m.uuid = pc.uuid
"""

In [56]:
async def process_fen(sample):
    player_color, uuid, move_index, fen = sample
    transport, engine = await chess.engine.popen_uci("/opt/homebrew/bin/stockfish")
    board = chess.Board(fen=fen)
    info = await engine.analyse(board, chess.engine.Limit(time=0.1))
    score = info["score"]
    # if player_color == 'White':
    #     score = score.white().score()
    # else:
    #     score = score.black().score()
    await engine.quit()
    return [uuid, move_index, score]

In [57]:
async def process_data():
    samples = db.sql(q).to_df().values.tolist()
    res = await asyncio.gather(*map(process_fen, samples))
    return res

In [58]:
res = await process_data()

In [59]:
pd.DataFrame(res, columns=['uuid', 'game_move_index', 'player_centipawn_score'])

,uuid,game_move_index,player_centipawn_score
0,0382938d-05f5-11ef-9338-b21a8d01000f,1,"PovScore(Cp(-36), BLACK)"
1,0382938d-05f5-11ef-9338-b21a8d01000f,2,"PovScore(Cp(+35), WHITE)"
2,0382938d-05f5-11ef-9338-b21a8d01000f,3,"PovScore(Cp(-26), BLACK)"
3,0382938d-05f5-11ef-9338-b21a8d01000f,4,"PovScore(Cp(+31), WHITE)"
4,0382938d-05f5-11ef-9338-b21a8d01000f,5,"PovScore(Cp(-33), BLACK)"
...,...,...,...
82,0382938d-05f5-11ef-9338-b21a8d01000f,83,"PovScore(Mate(-15), BLACK)"
83,0382938d-05f5-11ef-9338-b21a8d01000f,84,"PovScore(Mate(+2), WHITE)"
84,0382938d-05f5-11ef-9338-b21a8d01000f,85,"PovScore(Mate(-1), BLACK)"
85,0382938d-05f5-11ef-9338-b21a8d01000f,86,"PovScore(Mate(+1), WHITE)"
